## Prepare environment

In [1]:
!pip install git+https://github.com/katarinagresova/ensembl_scraper.git

  Cloning https://github.com/katarinagresova/ensembl_scraper.git to /tmp/pip-req-build-8ibftq7o
     |████████████████████████████████| 865 kB 4.2 MB/s eta 0:00:01
  Using cached twobitreader-3.1.7.tar.gz (9.2 kB)
Using legacy 'setup.py install' for scraper, since package 'wheel' is not installed.
Using legacy 'setup.py install' for twobitreader, since package 'wheel' is not installed.
    Running setup.py install for twobitreader ... done
    Running setup.py install for scraper ... done
You should consider upgrading via the '/home/petr/.pyenv/versions/3.8.9/envs/genomic_benchmarks/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
!wget https://raw.githubusercontent.com/katarinagresova/ensembl_scraper/main/requirements.txt

--2021-10-12 12:37:40--  https://raw.githubusercontent.com/katarinagresova/ensembl_scraper/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     337  --.-KB/s    in 0s      

2021-10-12 12:37:40 (11.7 MB/s) - ‘requirements.txt’ saved [337/337]



In [3]:
!pip install -r requirements.txt

  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached charset_normalizer-2.0.4-py3-none-any.whl (36 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
     |████████████████████████████████| 11.5 MB 4.2 MB/s eta 0:00:01
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 24.9 MB 1.5 MB/s eta 0:00:01
  Using cached tqdm-4.62.2-py2.py3-none-any.whl (76 kB)
  Using cached urllib3-1.26.6-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.10.8
    Uninstalling certifi-2021.10.8:
      Successfully uninstalled certifi-2021.10.8
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 2.0.6
    Uninstalling charset-normalizer-2.0.6:
      Successfully uninstalled charset-normalizer-2.0.6
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled 

## Create config file

In [5]:
import yaml

config = {
    "root_dir": "../../datasets/",
    "organisms": {
        "mus_musculus": {
            "external_feature"
        }
    }
}

user_config = 'user_config.yaml'
with open(user_config, 'w') as handle:
  yaml.dump(config, handle)

## Prepare directories

In [7]:
from pathlib import Path

BASE_FILE_PATH = Path("../../datasets/dummy_mouse_enhancers_ensembl/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)

## Run tool

In [6]:
!python -m scraper.ensembl_scraper -c user_config.yaml

Processing feature files:   0%|                           | 0/1 [00:00<?, ?it/s]INFO:root:download_file(): Going to download file from path ftp://ftp.ensembl.org/pub/release-100/mysql/regulation_mart_100/mmusculus_external_feature__external_feature__main.txt.gz
INFO:root:download_file(): File downloaded to path ../../datasets//tmp//mus_musculus_external_feature.txt.gz.
INFO:root:parse_feature_file(): Going to parse file ../../datasets//tmp//mus_musculus_external_feature.txt.gz
INFO:root:parse_feature_file(): Done parsing file ../../datasets//tmp//mus_musculus_external_feature.txt.gz


Processing feature types:   0%|                           | 0/1 [00:00<?, ?it/s]INFO:root:find_sequences_and_save_to_fasta(): Going to find sequences based on genomic loci.
INFO:root:download_2bit_file(): Going to download 2bit file mm10
INFO:root:download_2bit_file(): File for mm10 downloaded to path ../../datasets//tmp/mm10.2bit.
INFO:root:find_sequences_and_save_to_fasta(): Done finding sequences.
INFO

## Cleaning

In [8]:
!mv ../../datasets/mus_musculus_external_feature_enhancer ../../datasets/dummy_mouse_enhancers_ensembl/

In [9]:
!rm requirements.txt user_config.yaml

In [ ]:
!rm -rf ../../datasets/tmp/

## Final reformatting

  * gzip all CSV files
  * add extra formatting to yaml config file

In [11]:
!find ../../datasets/dummy_mouse_enhancers_ensembl/ -type f -name "*.csv" -exec gzip {} \;

In [12]:
with open("../../datasets/dummy_mouse_enhancers_ensembl/metadata.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz'}},
 'version': 0}

In [14]:
config['classes']['positive']['extra_processing'] = 'ENSEMBL_MOUSE_GENOME' 
config['classes']['negative']['extra_processing'] = 'ENSEMBL_MOUSE_GENOME' 

config

{'classes': {'negative': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_MOUSE_GENOME'},
  'positive': {'type': 'fa.gz',
   'url': 'ftp://ftp.ensembl.org/pub/release-100/fasta/mus_musculus/dna/Mus_musculus.GRCm38.dna_rm.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_MOUSE_GENOME'}},
 'version': 0}

In [15]:
with open("../../datasets/dummy_mouse_enhancers_ensembl/metadata.yaml", 'w') as handle:
  yaml.dump(config, handle)